In [1]:
#print("yo yo")
import warnings
warnings.filterwarnings('ignore')
!pip install ipywidgets
!pip install langchain langchain_community langchain_chroma

In [2]:
!pip install -qU langchain-groq
!pip install -U langchain-huggingface

In [ ]:
import getpass
import os 

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192", temperature = 0)

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(["https://admissions.wisc.edu/apply-as-a-freshman/", "https://admissions.wisc.edu/apply-as-a-transfer/", 
                        "https://admissions.wisc.edu/apply-as-a-reentry-student/", 
                        "https://admissions.wisc.edu/apply-as-a-second-degree-student/"]
)

docs = loader.load()
len(docs[0].page_content)

#docs


In [ ]:
len(docs)
print(docs[1].metadata)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=300)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# from langchain_core.prompts import PromptTemplate

# template = """You are an addmissions advisor at UW Madison, use the following pieces of context to aid your answer to the question at the end.
# If you don't know the answer, say that you don't know, but you recommend the student to consult wisc.edu and reach out to admissions or help offices.
# Use three sentences maximum and keep the answer as concise as possible.
# If the student has not specified what type of student they are, assume First-Year Student. 
# Always say "thanks for asking!" at the end of the answer.

# Context: {context}

# Question: {question}

# Helpful Answer:"""
# college_advisor = PromptTemplate.from_template(template)

In [ ]:
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | college_advisor
#     | llm
#     | StrOutputParser()
# )

# #rag_chain.invoke("What is RAG?")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
"""You are an addmissions advisor at UW Madison, use the following pieces of context to aid your answer to the question at the end.
If you don't know the answer, say that you don't know, but you recommend the student to consult wisc.edu and reach out to admissions or help offices.
Use three sentences maximum and keep the answer as concise as possible.
If the student has not specified what type of student they are, assume First-Year Student. 
Always say "thanks for asking!" at the end of the answer.

Context: {context}"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain2 = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
rag_chain.invoke("What are the Math requirnment for incoming freshman?")

In [ ]:
rag_chain.invoke("What English tests does UW Madison Accept for international students?")

In [ ]:
rag_chain.invoke("Is Duoloingo accepted?")

In [ ]:
rag_chain.invoke("early action deadline?")

In [ ]:
rag_chain.invoke("I missed the early application deadline, what are my options? Can I still reach out to someone?")

In [ ]:
rag_chain.invoke("What are my limitations if I am a second degree student?")